In [1]:
from requests import get
import pandas as pd
from bs4 import BeautifulSoup
from time import sleep
from random import randint
from time import time
from warnings import warn
from IPython.core.display import clear_output

Getting Data

In [2]:
years_url = [str(i) for i in range(1950,2019)]
years=[]
ranks=[]
artists=[]
songs=[]


# Time
start_time = time()
req = 0

# For every year in the interval 1950-2018
for year_url in years_url:

    # Make a get request
    response = get("http://billboardtop100of.com/"+ year_url +"-2/")
    

    # Pause the loop
    sleep(randint(8,15))

    # Monitor the requests
    req += 1
    elapsed_time = time() - start_time
    print('Req:{}; Frequency: {} req/s'.format(req, req/elapsed_time))
    clear_output(wait = True)
        # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Req: {}; Status code: {}'.format(req, response.status_code))

    # Parse the content of the request with BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')
    print(soup.title)
    
    # Select top 10 songs from each page
    #2017 & 2018 had a different html layout
    
    if year_url not in ["2017","2018"]:
        musicdata = soup.find_all("tr",limit = 10)

        for data in musicdata:
            years.append(soup.find("h1",class_="entry-title").text)

            ranks.append(data.contents[1].text.strip().split("\n")[0])

            artists.append(data.contents[3].text.strip().split("\n")[0])

            songs.append(data.contents[5].text.strip().split("\n")[0])
    
    else:
        for data in range(10):
            years.append(year_url)
            
            ranks.append(data+1)
            
            artists.append(soup.find_all("div",{"class":"ye-chart-item__artist"},limit = 10)[data].text)
            
            songs.append(soup.find_all("div",{"class":"ye-chart-item__title"},limit = 10)[data].text)

<title>2018 - Billboard Top 100</title>


Saving lists to a dataframe

In [3]:
df = pd.DataFrame({'Year':years,
                   'Rank':ranks,
                   'Artist':artists,
                   'Song':songs})

In [4]:
df["Year"] = pd.to_numeric(df["Year"])

In [7]:
df.to_csv('Top_10_Songs_1950-2018.csv')